# I. Выпуклость, необходимое условие минимума, двойственная задача

## 1. Критерий выпуклости второго порядка

Optimal allocation with resource constraint:
$$f_{i}\left(x_{i}\right) = a_{i}e^{x_{i}}, a_{i} > 0, i \in \{ 1, ..., n\}$$

Также ограничение бюджета:
$$\sum^{n}_{i=1}x_{i} = b, b \in R$$

Задача минимизации:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}$$

Итоговая задача:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}, \sum^{n}_{i=1}x_{i} = b, b \in R$$

Для изучения выпуклости рассмотрим гессиан оптимизируемой функции:
$$H(x)_{i,j} = 0, i \neq j$$
$$H(x)_{i,j} = a_{j}e^{x_{j}}, i = j$$
$$\text{При } a_{i} > 0 \text{ очевидно, что } H(x)_{i,i} = a_{i}e^{x_{i}} > 0, \text{ так как } \forall x \in R: e^{x_{i}} > 0$$
Матрица такого вида является положительно определенной.

Так же рассмотрим знакоопределенность гессиана ограничений:
$$H(x) = \textbf{0}$$ 

Таким образом, задача является выпуклой.

## 2. Условие оптимальности

Рассмотрим Лангранжиан системы:
$$L(x, \nu) = \sum^{n}_{i=1}a_{i}e^{x_{i}} + \nu (\textbf{1}^Tx-b)$$

Для потимальности найдем точки нуля производной лагранжиана, такие точки отметим звездой:
$$\nabla L (x^{\star}, \nu^{\star}) = \left( a_{1}e^{x_{1}^{\star}}, ... , a_{n}e^{x_{n}^{\star}} \right)^T + {\boldsymbol\nu^{\star}}^T  = 0$$

Выражая покомпонентно элементы вектора x*:
$$a_{i}e^{x_{i}^{\star}} = -\nu^{\star}, i \in \{ 1, ..., n\}$$
$$x_{i}^{\star} = ln\left( - \frac{\nu^{\star}}{a_{i}}\right), i \in \{ 1, ..., n\}$$

Таким образом условия оптимума ККТ будут иметь вид следующей системы:

\begin{cases}
  x_{i}^{\star} = ln\left( - \frac{\nu^{\star}}{a_{i}}\right), i \in \{ 1, ..., n\}\\
  \textbf{1}^Tx^{\star} = b
\end{cases}

## 3. Двойственная задача

Двойственной к оригинальной задачей будет инфинум лагранжиана по вектору переменных x.

$$g(\nu) = \underset{x}{inf} L(x, \nu)$$
$$g(\nu) = -n \nu + \nu \left( \sum^{n}_{i=1}ln\left( -\frac{\nu}{a_{i}}\right) -b \right) = \nu \left( \sum^{n}_{i=1}ln\left( -\frac{\nu}{a_{i}}\right) -n -b \right)$$

Тогда двойственная задача есть:
$$\nu \left( \sum^{n}_{i=1}ln\left( -\frac{\nu}{a_{i}}\right) -n -b \right) \rightarrow max$$

# II. Решение тестовых примеров

In [141]:
import numpy as np
import cvxpy as cp
import time
import polars as pl
import matplotlib.pyplot as plt
import os

## 1. Генерация примеров

In [142]:
n_dims = range(10, 110, 10)
n_examples = 100
n_seeds = 100

In [143]:
raw_experiment_sheet = pl.DataFrame({"dim": n_dims}).select(
    pl.all().repeat_by(n_examples).flatten()
)

In [144]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=0.1, high=10, size=dim_),
        return_dtype=pl.List(float),
        returns_scalar=False,
    )
    .alias("a"),
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=-10, high=10, size=1),
        return_dtype=float,
        returns_scalar=True,
    )
    .alias("b"),
)

In [145]:
raw_experiment_sheet

dim,a,b
i64,list[f64],f64
10,"[2.493259, 7.169756, … 8.358499]",-7.459575
10,"[3.557954, 1.303087, … 7.090329]",-6.781608
10,"[5.089566, 2.735855, … 0.254117]",-9.201723
10,"[2.642611, 7.524052, … 8.857005]",3.320509
10,"[9.975331, 0.425117, … 8.308354]",-0.691465
…,…,…
100,"[6.339542, 0.135605, … 8.041473]",-2.834098
100,"[7.724265, 2.763721, … 1.957112]",-0.778715
100,"[1.423904, 9.421519, … 1.857509]",-2.019832


## 2. Решение солвером CVXPY

In [146]:
def solve_oawrc(a, b):

    a = np.asarray(a)

    x = cp.Variable(a.shape[0])

    objective = cp.Minimize(cp.sum(a @ cp.exp(x)))

    constraints = [cp.sum(x) - b == 0]

    prob = cp.Problem(objective, constraints)

    try:
        start_t = time.time()
        result = prob.solve()
        sovle_time = time.time() - start_t
    except:
        res = {"cvxpy_x": None, "cvxpy_y": None, "cvxpy_t": None}
    else:
        res = {"cvxpy_x": list(x.value), "cvxpy_y": result, "cvxpy_t": sovle_time}

    return res

In [147]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b")
        .map_elements(
            lambda x: solve_oawrc(**x),
            return_dtype=pl.Struct(
                {"cvxpy_x": pl.List(float), "cvxpy_y": float, "cvxpy_t": float}
            ),
        )
        .alias("cvxpy_out")
    )
).unnest("cvxpy_out")

In [148]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t
i64,list[f64],f64,list[f64],f64,f64
10,"[2.493259, 7.169756, … 8.358499]",-7.459575,"[-0.331797, -1.388131, … -1.541524]",17.893207,0.00525
10,"[3.557954, 1.303087, … 7.090329]",-6.781608,"[-0.617977, 0.386464, … -1.307528]",19.178641,0.003252
10,"[5.089566, 2.735855, … 0.254117]",-9.201723,"[-1.212595, -0.591848, … 1.784458]",15.137386,0.002485
10,"[2.642611, 7.524052, … 8.857005]",3.320509,"[0.430925, -0.615437, … -0.778542]",40.659835,0.002588
10,"[9.975331, 0.425117, … 8.308354]",-0.691465,"[-1.07519, 2.080394, … -0.892341]",34.039601,0.002499
…,…,…,…,…,…
100,"[6.339542, 0.135605, … 8.041473]",-2.834098,"[-0.5982, 3.246568, … -0.835997]",348.545572,0.003595
100,"[7.724265, 2.763721, … 1.957112]",-0.778715,"[-0.718903, 0.308924, … 0.653943]",376.40253,0.00358
100,"[1.423904, 9.421519, … 1.857509]",-2.019832,"[0.986949, -0.902371, … 0.721109]",382.039058,0.003087


## 3. Решение прямой задачи методом Ньютона 

In [149]:
def random_sum_bound(dim, b):
    rnd = np.random.uniform(size=dim)
    return list(rnd / rnd.sum() * b)

In [150]:
raw_experiment_sheet = raw_experiment_sheet.select(
    pl.all().repeat_by(n_seeds).flatten()
).with_columns(
    pl.struct("dim", "b")
    .map_elements(lambda x: random_sum_bound(**x), return_dtype=pl.List(float))
    .alias("x_init")
)
raw_experiment_sheet = raw_experiment_sheet.with_columns(pl.lit(np.random.uniform(size=len(raw_experiment_sheet))).alias("nu_init"))

In [151]:
raw_experiment_sheet = raw_experiment_sheet.drop_nans()

In [152]:
def f(x, a):
    return np.dot(a, np.exp(x)).reshape(1,1)

def grad_f(x, a):
    return (a * np.exp(x)).reshape(1,-1)

def hessian_f(x, a):
    return np.diag(a * np.exp(x))

def g(x, b):
    return np.array([np.sum(np.asarray(x)) - b]).reshape(1,1)

def grad_g(x):
    return np.ones(len(x)).reshape(1,-1)

def hessian_g(x):
    return np.zeros([len(x)]*2)

In [153]:
def solve_oawrc_newton(a, b, x_init, nu_init, cvxpy_y, eps=0.01, max_iters=10000):
    sovle_time = 0
    newton_hist_y = []
    newton_hist_ops = []

    x = np.asarray(x_init)
    a = np.asarray(a)
    nu = nu_init
    
    for i in range(max_iters):
        iter_ops = 0
        start_t_iter = time.time()

        grad_L = grad_f(x, a) - grad_g(x) * nu
        hessian_L = hessian_f(x, a)

        iter_ops += int(3*grad_L.shape[-1])

        hessian_L_full = np.block([
            [hessian_L, grad_g(x).T],
            [grad_g(x), np.zeros([1,1])]
        ])
        
        grad_L_full = np.concatenate([grad_L, g(x, b).reshape(1,-1)], axis = -1)
        d = np.linalg.solve(hessian_L_full, -grad_L_full.T)

        iter_ops += int(hessian_L_full.shape[0]**3)

        x += d[:len(x),0]
        nu += d[len(x):,0]

        iter_ops += d.shape[0]

        sovle_time += time.time() - start_t_iter
        newton_hist_y.append(float(f(x, a)[0]))
        newton_hist_ops.append(int(i*iter_ops))

        if np.abs(f(x, a)[0] - cvxpy_y) < eps:
            break
    
    return {"newton_x": x.tolist(), "newton_y": f(x, a)[0], "newton_t": sovle_time, "newton_hist_ops": newton_hist_ops, "newton_hist_y": newton_hist_y}

In [154]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b", "x_init", "nu_init", "cvxpy_y")
        .map_elements(
            lambda x: solve_oawrc_newton(**x),
            return_dtype=pl.Struct(
                {"newton_x": pl.List(float), "newton_y": float, "newton_t": float, "newton_hist_ops": pl.List(int), "newton_hist_y": pl.List(float)}
            ),
        )
        .alias("newton_out")
    )
).unnest("newton_out")

/tmp/ipykernel_20096/2671493511.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  newton_hist_y.append(float(f(x, a)[0]))


In [155]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t,x_init,nu_init,newton_x,newton_y,newton_t,newton_hist_ops,newton_hist_y
i64,list[f64],f64,list[f64],f64,f64,list[f64],f64,list[f64],f64,f64,list[i64],list[f64]
10,"[2.493259, 7.169756, … 8.358499]",-7.459575,"[-0.331797, -1.388131, … -1.541524]",17.893207,0.00525,"[-1.335779, -0.522027, … -0.325977]",0.678465,"[-0.315413, -1.396625, … -1.55411]",17.897613,0.000421,"[0, 1372, 2744]","[20.475218, 18.165151, 17.897613]"
10,"[2.493259, 7.169756, … 8.358499]",-7.459575,"[-0.331797, -1.388131, … -1.541524]",17.893207,0.00525,"[-0.687726, -1.066422, … -0.130056]",0.655194,"[-0.332615, -1.388934, … -1.542329]",17.893271,0.000187,"[0, 1372, … 4116]","[21.67056, 18.351807, … 17.893271]"
10,"[2.493259, 7.169756, … 8.358499]",-7.459575,"[-0.331797, -1.388131, … -1.541524]",17.893207,0.00525,"[-0.910784, -0.153179, … -0.056987]",0.529198,"[-0.338265, -1.394645, … -1.543084]",17.894702,0.000127,"[0, 1372, 2744]","[20.50199, 18.027724, 17.894702]"
10,"[2.493259, 7.169756, … 8.358499]",-7.459575,"[-0.331797, -1.388131, … -1.541524]",17.893207,0.00525,"[-1.092097, -0.268626, … -0.565674]",0.292508,"[-0.338301, -1.394666, … -1.548208]",17.896859,0.000164,"[0, 1372, … 4116]","[26.757062, 19.454891, … 17.896859]"
10,"[2.493259, 7.169756, … 8.358499]",-7.459575,"[-0.331797, -1.388131, … -1.541524]",17.893207,0.00525,"[-0.423041, -1.201635, … -0.206343]",0.212297,"[-0.333914, -1.389384, … -1.54159]",17.893324,0.000159,"[0, 1372, 2744]","[19.437801, 17.934175, 17.893324]"
…,…,…,…,…,…,…,…,…,…,…,…,…
100,"[5.13129, 3.181561, … 7.639559]",-2.782651,"[-0.277782, 0.200068, … -0.675871]",388.621039,0.005188,"[-0.040527, -0.013429, … -0.040299]",0.197275,"[-0.278079, 0.199907, … -0.676061]",388.621508,0.001018,"[0, 1030702, … 14429828]","[841788.006746, 309878.273767, … 388.621508]"
100,"[5.13129, 3.181561, … 7.639559]",-2.782651,"[-0.277782, 0.200068, … -0.675871]",388.621039,0.005188,"[-0.015799, -0.032692, … -0.019232]",0.132196,"[-0.278028, 0.199957, … -0.676011]",388.621252,0.00102,"[0, 1030702, … 14429828]","[753258.885155, 277309.983105, … 388.621252]"
100,"[5.13129, 3.181561, … 7.639559]",-2.782651,"[-0.277782, 0.200068, … -0.675871]",388.621039,0.005188,"[-0.016878, -0.00459, … -0.048505]",0.942385,"[-0.277992, 0.199993, … -0.675975]",388.62113,0.001034,"[0, 1030702, … 14429828]","[674804.202171, 248448.317004, … 388.62113]"


# III. Анализ зависимостей

# IV. Выводы